# The Task

In [96]:
import bq_helper
from bq_helper import BigQueryHelper
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="credentials.json"
bq_assistant = BigQueryHelper("bigquery-public-data", "stackoverflow")

QUERY = "SELECT q.id, q.title, q.body, q.tags, a.body as answers, a.score FROM `bigquery-public-data.stackoverflow.posts_questions` AS q INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a ON q.id = a.parent_id WHERE q.tags LIKE '%python%' LIMIT 500000"
df = bq_assistant.query_to_pandas(QUERY)
df.to_csv('data/Original_data.csv')

In [1]:
import pandas as pd
import numpy as np
import spacy
EN = spacy.load('en_core_web_sm')
df = pd.read_csv('data/Original_data.csv')
df.head()

id                                              title  \
0  32490721          What does the /= operator mean in Python?   
1  16784162  What is a solid example of something that can ...   
2   2763750  How to replace only part of the match with pyt...   
3   7931726          Are datagrams always received completely?   
4   3838329         How can I check if two segments intersect?   

                                                body  \
0  <p>What does the operator <code>/=</code> (sla...   
1  <p>I've heard from many Pythonists that they p...   
2  <p>I need to match two cases by one reg expres...   
3  <p>Most datagram receiving functions such as c...   
4  <p>How can I check if 2 segments intersect?</p...   

                                                tags  \
0                                      python|syntax   
1  python|haskell|clojure|functional-programming|...   
2                                       python|regex   
3                         java|python|c|udp|datagram   
4                                        python|math   

                                             answers  score  
0  <p>It's an assignment operator shorthand for <...     35  
1  <p>As has been said, everything you can do wit...     20  
2  <pre><code> re.sub(r'(?:_a)?\.([^.]*)$', r'_su...     22  
3  <p>UDP datagrams cannot be partially delivered...     14  
4  <p>The equation of a line is:</p>\n\n<pre><cod...     42

In [2]:
print('Datebase shape:' + str(df.shape))

Datebase shape:(500000, 6)


In [3]:
df.isna().sum()

id         0
title      0
body       0
tags       0
answers    0
score      0
dtype: int64

In [ ]:
# concat answers for each post
aggregations = {
    'answers':{
        'combined_answers': lambda x: "\n".join(x)
    },
    'score':{
        'combined_score': 'sum'
    }
}
grouped = df.groupby(['id','title', 'body','tags'],as_index=False).agg(aggregations)
deduped_df = pd.DataFrame(grouped)

In [100]:
deduped_df.head()

id                                              title  \
                                                             
0   469  How can I find the full path to a font from it...   
1   594     cx_Oracle: How do I iterate over a result set?   
2   742                              Class views in Django   
3   766                                   Python and MySQL   
4  1476      How do you express binary literals in Python?   

                                                body  \
                                                       
0  <p>I am using the Photoshop's javascript API t...   
1  <p>There are several ways to iterate over a re...   
2  <p><a href="http://www.djangoproject.com/" rel...   
3  <p>I can get Python to work with Postgresql bu...   
4  <p>How do you express an integer as a binary n...   

                                    tags  \
                                           
0           python|macos|fonts|photoshop   
1   python|sql|database|oracle|cx-oracle   
2                python|django|views|oop   
3         python|mysql|postgresql|bpgsql   
4  python|syntax|binary|integer|literals   

                                             answers          score  
                                    combined_answers combined_score  
0  <p>open up a terminal (Applications->Utilities...             39  
1  <p>The canonical way is to use the built-in cu...             68  
2  <p>I needed to use class based views, but I wa...             75  
3  <p>I uploaded it and got an internal error</p>...             48  
4  <pre><code>&gt;&gt;&gt; print int('01010101111...            352

In [101]:
print('Max score before: ') 
print(np.max(df.score.values))

print('Max score after: ') 
print(np.max(deduped_df.score.values))

Max score before: 
5440
Max score after: 
9730


In [102]:
import re
import nltk
import inflect
from nltk.corpus import stopwords

def tokenize_text(text):
    "Apply tokenization using spacy to docstrings."
    tokens = EN.tokenizer(text)
    return [token.text.lower() for token in tokens if not token.is_space]

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def normalize(words):
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

def tokenize_code(text):
    "A very basic procedure for tokenizing code strings."
    return RegexpTokenizer(r'\w+').tokenize(text)

def preprocess_text(text):
    return ' '.join(normalize(tokenize_text(text)))

In [103]:
from bs4 import BeautifulSoup
from textblob import TextBlob

title_list = [] 
content_list = []
url_list = []
comment_list = []
sentiment_polarity_list = []
sentiment_subjectivity_list = []
vote_list =[]
tag_list = []
corpus_list = []

for i, row in deduped_df.iterrows():
    title_list.append(row.title.values[0])    # Get question title
    tag_list.append(row.tags.values[0])     # Get question tags
    
    # Questions
    content = row.body.values[0]
    soup = BeautifulSoup(content, 'lxml')
    if soup.code: soup.code.decompose()     # Remove the code section
    tag_p = soup.p
    tag_pre = soup.pre
    text = ''
    if tag_p: text = text + tag_p.get_text()
    if tag_pre: text = text + tag_pre.get_text()
        
    content_list.append(str(row.title.values[0]) + ' ' + str(text))   # Append title and question body data to the updated question body
    
    url_list.append('https://stackoverflow.com/questions/' + str(row.id.values[0]))
    
    # Answers
    content = row.answers.values[0]
    soup = BeautifulSoup(content, 'lxml')
    if soup.code: soup.code.decompose()
    tag_p = soup.p
    tag_pre = soup.pre
    text = ''
    if tag_p: text = text + tag_p.get_text()
    if tag_pre: text = text + tag_pre.get_text()
    comment_list.append(text)
    
    vote_list.append(row.score.values[0])       # Append votes
    
    corpus_list.append(content_list[-1] + ' ' + comment_list[-1])     # Combine the updated body and answers to make the corpus
    
    sentiment = TextBlob(row.answers.values[0]).sentiment
    sentiment_polarity_list.append(sentiment.polarity)
    sentiment_subjectivity_list.append(sentiment.subjectivity)

content_token_df = pd.DataFrame({'original_title': title_list, 'post_corpus': corpus_list, 'question_content': content_list, 'question_url': url_list, 'tags': tag_list, 'overall_scores':vote_list,'answers_content': comment_list, 'sentiment_polarity': sentiment_polarity_list, 'sentiment_subjectivity':sentiment_subjectivity_list})

In [105]:
content_token_df.head()

original_title  \
0  How can I find the full path to a font from it...   
1     cx_Oracle: How do I iterate over a result set?   
2                              Class views in Django   
3                                   Python and MySQL   
4      How do you express binary literals in Python?   

                                         post_corpus  \
0  How can I find the full path to a font from it...   
1  cx_Oracle: How do I iterate over a result set?...   
2  Class views in Django Django view points to a ...   
3  Python and MySQL I can get Python to work with...   
4  How do you express binary literals in Python? ...   

                                    question_content  \
0  How can I find the full path to a font from it...   
1  cx_Oracle: How do I iterate over a result set?...   
2  Class views in Django Django view points to a ...   
3  Python and MySQL I can get Python to work with...   
4  How do you express binary literals in Python? ...   

                               question_url  \
0   https://stackoverflow.com/questions/469   
1   https://stackoverflow.com/questions/594   
2   https://stackoverflow.com/questions/742   
3   https://stackoverflow.com/questions/766   
4  https://stackoverflow.com/questions/1476   

                                    tags  overall_scores  \
0           python|macos|fonts|photoshop              39   
1   python|sql|database|oracle|cx-oracle              68   
2                python|django|views|oop              75   
3         python|mysql|postgresql|bpgsql              48   
4  python|syntax|binary|integer|literals             352   

                                     answers_content  sentiment_polarity  \
0  open up a terminal (Applications->Utilities->T...            0.116667   
1  The canonical way is to use the built-in curso...            0.010082   
2  I needed to use class based views, but I wante...            0.080339   
3            I uploaded it and got an internal error            0.187500   
4                                       Another way.           -0.064497   

   sentiment_subjectivity  
0                0.554167  
1                0.423400  
2                0.439853  
3                0.390675  
4                0.605969

In [106]:
content_token_df.tags = content_token_df.tags.apply(lambda x: x.split('|'))   # Convert raw text data of tags into lists

# Make a dictionary to count the frequencies for all tags
tag_freq_dict = {}
for tags in content_token_df.tags:
    for tag in tags:
        if tag not in tag_freq_dict:
            tag_freq_dict[tag] = 0
        else:
            tag_freq_dict[tag] += 1

In [107]:
import heapq
most_common_tags = heapq.nlargest(20, tag_freq_dict, key=tag_freq_dict.get)

In [108]:
most_common_tags

['python',
 'python-3.x',
 'django',
 'pandas',
 'python-2.7',
 'numpy',
 'list',
 'matplotlib',
 'dictionary',
 'regex',
 'dataframe',
 'tkinter',
 'string',
 'csv',
 'flask',
 'arrays',
 'tensorflow',
 'json',
 'beautifulsoup',
 'selenium']

In [109]:
final_indices = []
for i,tags in enumerate(content_token_df.tags.values.tolist()):
    if len(set(tags).intersection(set(most_common_tags)))>1:   # The minimum length for common tags should be 2 because 'python' is a common tag for all
        final_indices.append(i)

In [110]:
final_data = content_token_df.iloc[final_indices]

In [111]:
import spacy
EN = spacy.load('en_core_web_sm')

# Preprocess text for 'question_body', 'post_corpus' and a new column 'processed_title'
final_data.question_content = final_data.question_content.apply(lambda x: preprocess_text(x))
final_data.post_corpus = final_data.post_corpus.apply(lambda x: preprocess_text(x))
final_data['processed_title'] = final_data.original_title.apply(lambda x: preprocess_text(x))

# Normalize numeric data for the scores
final_data.overall_scores = (final_data.overall_scores - final_data.overall_scores.mean()) / (final_data.overall_scores.max() - final_data.overall_scores.min())


KeyboardInterrupt



In [ ]:
final_data.tags = final_data.tags.apply(lambda x: '|'.join(x))    # Combine the lists back into text data
final_data.drop(['answers_content'], axis=1)     # Remove the answers_content columns because it is alreaady included in the corpus

In [ ]:
# Save the data
final_data.to_csv('data/Preprocessed_data.csv', index=False)